# MOE Megablocks - 124M, 350M, 40M and 5M

In [ ]:
# 1. Delete all outputs
!rm -rf out-baseline out-moe out-*

# 2. Keep tokenized data cache (don't delete this!)
!ls -lh tokenized_cache/

# 3. Fresh clone from GitHub to get clean code
%cd /content
!rm -rf bdml-p4
!git clone https://github.com/VJagz24/bdml-p4.git
%cd bdml-p4

print("✓ Fresh code from GitHub!")
print("✓ Tokenized cache preserved!")

total 8.0K
-rw-r--r-- 1 root root 3.9K Dec 11 15:58 train_tokenized_1024.pt
-rw-r--r-- 1 root root 3.9K Dec 11 15:58 val_tokenized_1024.pt
/content
Cloning into 'bdml-p4'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 33 (delta 17), reused 22 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 19.16 KiB | 19.16 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/bdml-p4
✓ Fresh code from GitHub!
✓ Tokenized cache preserved!


In [ ]:
# Install dependencies
!pip install -q megablocks
!pip install -q git+https://github.com/tgale96/grouped_gemm@main

# Configure to 124M + 5000 iters
!sed -i 's/n_layer = 4/n_layer = 12/g' train_baseline.py train_moe.py
!sed -i 's/n_head = 4/n_head = 12/g' train_baseline.py train_moe.py
!sed -i 's/n_embd = 256/n_embd = 768/g' train_baseline.py train_moe.py
!sed -i 's/block_size = 256/block_size = 1024/g' train_baseline.py train_moe.py
!sed -i 's/max_seq_length = 256/max_seq_length = 1024/g' train_baseline.py train_moe.py

# 5000 iterations
!sed -i 's/max_iters = 1000/max_iters = 5000/g' train_baseline.py train_moe.py

# More warmup
!sed -i 's/warmup_iters = 100/warmup_iters = 500/g' train_baseline.py train_moe.py

# MoE Top-2
!sed -i 's/top_k = 1/top_k = 2/g' train_moe.py

# Fix cache loading
!sed -i 's/torch.load(train_cache_file)/torch.load(train_cache_file, weights_only=False)/g' train_baseline.py train_moe.py
!sed -i 's/torch.load(val_cache_file)/torch.load(val_cache_file, weights_only=False)/g' train_baseline.py train_moe.py

# Reduce MoE batch size for memory
!sed -i 's/batch_size = 8/batch_size = 4/g' train_moe.py
!sed -i 's/gradient_accumulation_steps = 4/gradient_accumulation_steps = 2/g' train_moe.py

print("✓ Configuration complete!")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✓ Configuration complete!


In [ ]:
print("=== BASELINE ===")
!grep "n_layer\|n_head\|n_embd\|block_size\|max_iters\|batch_size" train_baseline.py | head -8

print("\n=== MOE ===")
!grep "n_layer\|n_head\|n_embd\|block_size\|max_iters\|batch_size\|top_k" train_moe.py | head -9

=== BASELINE ===
n_layer = 12
n_head = 12
n_embd = 768
block_size = 1024
batch_size = 8
max_iters = 5000     # Total training iterations
lr_decay_iters = max_iters
    print(f"Model: {n_layer}L, {n_head}H, {n_embd}D")

=== MOE ===
n_layer = 12
n_head = 12
n_embd = 768
block_size = 1024
top_k = 2
batch_size = 4
max_iters = 5000     # Total training iterations
lr_decay_iters = max_iters
    print(f"Model: {n_layer}L, {n_head}H, {n_embd}D (MoE: {num_experts} experts, top_k={top_k})")


In [ ]:
# Verify MoE layer config
!grep -A 5 "ffn_hidden_size" model.py

            ffn_hidden_size=4 * config.n_embd,
            moe_num_experts=config.num_experts,
            moe_top_k=config.top_k,
            num_layers=1,
            bias=config.bias,
            fp16=False,


In [ ]:
# Baseline first
print("="*70)
print("BASELINE 124M - 5000 ITERATIONS - FRESH START")
print("="*70)
!python train_baseline.py

BASELINE 124M - 5000 ITERATIONS - FRESH START
BASELINE GPT TRAINING
Device: cuda
Dtype: bfloat16
Model: 12L, 12H, 768D
Batch size: 8 × 4 = 32
Max iterations: 5000

Loading dataset...
✗ Cache not found, tokenizing dataset (this will take ~17 minutes)...
Train examples: 1,801,350
Val examples: 3,760
Tokenizing train dataset...
Tokenizing train: 100% 1801350/1801350 [31:35<00:00, 950.28 examples/s]
Tokenizing validation dataset...
Tokenizing val: 100% 3760/3760 [00:04<00:00, 831.56 examples/s]
Saving tokenized dataset to cache...
✓ Cache saved!
✓ Data loaded: 225169 train batches, 470 val batches
  Data loading time: 1906.48s

Initializing model...
number of parameters: 123.69M
  Model initialization time: 2.87s
Setting up optimizer...

STARTING TRAINING

step 0: train loss 10.5150, val loss 10.5244
iter 0: loss 10.5015, time 3849.21ms, mfu -100.00%
iter 10: loss 0.9437, time 275.99ms, mfu 32.55%
iter 20: loss 0.5137, time 279.55ms, mfu 32.51%
iter 30: loss 0.3567, time 276.64ms, mfu 32.5

In [ ]:
# Then MoE
print("="*70)
print("MOE 124M - 5000 ITERATIONS - TOP-2 - FRESH START")
print("="*70)
!python train_moe.py

MOE 124M - 5000 ITERATIONS - TOP-2 - FRESH START
MOE GPT TRAINING
Device: cuda
Dtype: bfloat16
Model: 12L, 12H, 768D (MoE: 8 experts, top_k=2)
Batch size: 4 × 2 = 8
Max iterations: 5000

Loading dataset...
✓ Loading tokenized dataset from cache...
Train examples: 1,801,350
Val examples: 3,760
✓ Data loaded: 450338 train batches, 940 val batches
  Data loading time: 2.77s

Initializing model...
number of parameters: 520.09M
  Model initialization time: 1.90s
Setting up optimizer...

STARTING TRAINING

step 0: train loss 9.7961, val loss 9.8062
iter 0: loss 9.8010, time 6631.52ms, mfu -100.00%
iter 10: loss 5.0493, time 132.95ms, mfu 63.86%
iter 20: loss 1.0159, time 131.59ms, mfu 63.93%
iter 30: loss 0.6735, time 128.14ms, mfu 64.16%
iter 40: loss 0.7642, time 132.80ms, mfu 64.14%
iter 50: loss 0.6339, time 130.46ms, mfu 64.23%
iter 60: loss 0.7333, time 131.18ms, mfu 64.28%
iter 70: loss 0.4121, time 131.92ms, mfu 64.29%
iter 80: loss 0.5004, time 131.08ms, mfu 64.34%
iter 90: loss 0.3

In [ ]:
# Reduce baseline batch size to match MoE
!sed -i 's/batch_size = 8/batch_size = 4/g' train_baseline.py
!sed -i 's/gradient_accumulation_steps = 4/gradient_accumulation_steps = 2/g' train_baseline.py

# Verify
print("=== BASELINE (Updated) ===")
!grep "batch_size\|gradient_accumulation" train_baseline.py | head -2

print("\n=== MOE (Same) ===")
!grep "batch_size\|gradient_accumulation" train_moe.py | head -2

# Delete old baseline output
!rm -rf out-baseline

# Retrain baseline with batch=4×2=8
print("\n" + "="*70)
print("RETRAINING BASELINE WITH SAME BATCH SIZE AS MOE")
print("Batch: 4 × 2 = 8 (matching MoE)")
print("="*70)

!python train_baseline.py

=== BASELINE (Updated) ===
batch_size = 4
gradient_accumulation_steps = 2  # Effective batch size = 8 * 4 = 32

=== MOE (Same) ===
batch_size = 4
gradient_accumulation_steps = 2  # Effective batch size = 8 * 4 = 32

RETRAINING BASELINE WITH SAME BATCH SIZE AS MOE
Batch: 4 × 2 = 8 (matching MoE)
BASELINE GPT TRAINING
Device: cuda
Dtype: bfloat16
Model: 12L, 12H, 768D
Batch size: 4 × 2 = 8
Max iterations: 5000

Loading dataset...
✓ Loading tokenized dataset from cache...
Train examples: 1,801,350
Val examples: 3,760
✓ Data loaded: 450338 train batches, 940 val batches
  Data loading time: 2.72s

Initializing model...
number of parameters: 123.69M
  Model initialization time: 2.91s
Setting up optimizer...

STARTING TRAINING

step 0: train loss 10.5149, val loss 10.5195
iter 0: loss 10.4792, time 2441.22ms, mfu -100.00%
iter 10: loss 0.6548, time 80.40ms, mfu 27.93%
iter 20: loss 0.4953, time 80.59ms, mfu 27.93%
iter 30: loss 0.4469, time 80.36ms, mfu 27.93%
iter 40: loss 0.7058, time 83.6

In [ ]:
# Configure to 350M (GPT-2 Medium)
!sed -i 's/n_layer = 12/n_layer = 24/g' train_baseline.py train_moe.py
!sed -i 's/n_head = 12/n_head = 16/g' train_baseline.py train_moe.py
!sed -i 's/n_embd = 768/n_embd = 1024/g' train_baseline.py train_moe.py

# Keep everything else same
# block_size = 1024 (already set)
# max_iters = 5000 (already set)
# batch_size = 4 for BOTH (already set)
# gradient_accumulation_steps = 2 for BOTH (already set)

print("✓ Updated to 350M configuration")

✓ Updated to 350M configuration


In [ ]:
# Further reduce batch size for 350M
!sed -i 's/batch_size = 4/batch_size = 2/g' train_baseline.py train_moe.py
!sed -i 's/gradient_accumulation_steps = 2/gradient_accumulation_steps = 2/g' train_baseline.py train_moe.py

# Effective batch = 2 × 2 = 4 (very small but will fit!)

print("✓ Reduced batch to 2×2=4 for memory safety")

✓ Reduced batch to 2×2=4 for memory safety


In [ ]:
print("=== BASELINE 350M ===")
!grep "n_layer\|n_head\|n_embd\|batch_size\|gradient_accumulation\|max_iters" train_baseline.py | head -8

print("\n=== MOE 350M ===")
!grep "n_layer\|n_head\|n_embd\|batch_size\|gradient_accumulation\|max_iters\|top_k" train_moe.py | head -9

=== BASELINE 350M ===
n_layer = 24
n_head = 16
n_embd = 1024
batch_size = 2
gradient_accumulation_steps = 2  # Effective batch size = 8 * 4 = 32
max_iters = 5000     # Total training iterations
lr_decay_iters = max_iters
    print(f"Model: {n_layer}L, {n_head}H, {n_embd}D")

=== MOE 350M ===
n_layer = 24
n_head = 16
n_embd = 1024
top_k = 2
batch_size = 2
gradient_accumulation_steps = 2  # Effective batch size = 8 * 4 = 32
max_iters = 5000     # Total training iterations
lr_decay_iters = max_iters
    print(f"Model: {n_layer}L, {n_head}H, {n_embd}D (MoE: {num_experts} experts, top_k={top_k})")


In [ ]:
!rm -rf out-baseline out-moe
print("✓ Cleaned outputs")

✓ Cleaned outputs


In [ ]:
import time
start_time = time.time()

print("="*70)
print("BASELINE 350M - 5000 ITERATIONS")
print("Batch: 2 × 2 = 4")
print("Expected time: ~90-120 minutes")
print("="*70)

!python train_baseline.py

elapsed = (time.time() - start_time) / 60
print(f"\n✓ Baseline completed in {elapsed:.1f} minutes")

BASELINE 350M - 5000 ITERATIONS
Batch: 2 × 2 = 4
Expected time: ~90-120 minutes
BASELINE GPT TRAINING
Device: cuda
Dtype: bfloat16
Model: 24L, 16H, 1024D
Batch size: 2 × 2 = 4
Max iterations: 5000

Loading dataset...
✓ Loading tokenized dataset from cache...
Train examples: 1,801,350
Val examples: 3,760
✓ Data loaded: 900675 train batches, 1880 val batches
  Data loading time: 10.87s

Initializing model...
number of parameters: 353.82M
  Model initialization time: 7.31s
Setting up optimizer...

STARTING TRAINING

step 0: train loss 9.9041, val loss 9.9302
iter 0: loss 9.9073, time 2775.07ms, mfu -100.00%
iter 10: loss 0.5065, time 121.10ms, mfu 26.29%
iter 20: loss 0.4896, time 119.27ms, mfu 26.33%
iter 30: loss 0.2348, time 123.65ms, mfu 26.27%
iter 40: loss 0.1365, time 119.14ms, mfu 26.32%
iter 50: loss 0.1231, time 124.18ms, mfu 26.25%
iter 60: loss 0.0291, time 120.34ms, mfu 26.27%
iter 70: loss 0.3410, time 124.12ms, mfu 26.21%
iter 80: loss 0.2396, time 120.03ms, mfu 26.24%
iter

In [ ]:
start_time = time.time()

print("="*70)
print("MOE 350M (~1.5B params) - 5000 ITERATIONS - TOP-2")
print("Batch: 2 × 2 = 4")
print("Expected time: ~120-180 minutes")
print("="*70)

!python train_moe.py

elapsed = (time.time() - start_time) / 60
print(f"\n✓ MoE completed in {elapsed:.1f} minutes")

MOE 350M (~1.5B params) - 5000 ITERATIONS - TOP-2
Batch: 2 × 2 = 4
Expected time: ~120-180 minutes
MOE GPT TRAINING
Device: cuda
Dtype: bfloat16
Model: 24L, 16H, 1024D (MoE: 8 experts, top_k=2)
Batch size: 2 × 2 = 4
Max iterations: 5000

Loading dataset...
✓ Loading tokenized dataset from cache...
Train examples: 1,801,350
Val examples: 3,760
✓ Data loaded: 900675 train batches, 1880 val batches
  Data loading time: 6.46s

Initializing model...
number of parameters: 1763.21M
  Model initialization time: 3.62s
Setting up optimizer...

STARTING TRAINING

step 0: train loss 10.5337, val loss 10.5255
iter 0: loss 10.5418, time 9946.35ms, mfu -100.00%
iter 10: loss 2.8520, time 225.89ms, mfu 63.24%
iter 20: loss 1.3215, time 227.78ms, mfu 63.19%
iter 30: loss 0.2733, time 231.43ms, mfu 63.04%
iter 40: loss 0.0008, time 231.00ms, mfu 62.92%
iter 50: loss 0.4663, time 223.60ms, mfu 63.02%
iter 60: loss 0.3930, time 231.53ms, mfu 62.88%
iter 70: loss 0.1764, time 231.10ms, mfu 62.78%
iter 80: 

In [ ]:
# Set to 40M configuration
!sed -i 's/n_layer = 24/n_layer = 6/g' train_baseline.py train_moe.py
!sed -i 's/n_head = 16/n_head = 6/g' train_baseline.py train_moe.py
!sed -i 's/n_embd = 1024/n_embd = 384/g' train_baseline.py train_moe.py
!sed -i 's/block_size = 1024/block_size = 512/g' train_baseline.py train_moe.py
!sed -i 's/max_seq_length = 1024/max_seq_length = 512/g' train_baseline.py train_moe.py

# Keep 5000 iterations
# Keep batch = 4×2=8 (should fit fine)
!sed -i 's/batch_size = 2/batch_size = 4/g' train_baseline.py train_moe.py

# Fix cache loading for 512 seq length
!sed -i 's/torch.load(train_cache_file)/torch.load(train_cache_file, weights_only=False)/g' train_baseline.py train_moe.py
!sed -i 's/torch.load(val_cache_file)/torch.load(val_cache_file, weights_only=False)/g' train_baseline.py train_moe.py

print("✓ Configured for 40M (6L, 6H, 384D, 512 seq)")

✓ Configured for 40M (6L, 6H, 384D, 512 seq)


In [ ]:
!rm -rf out-baseline out-moe

print("="*70)
print("BASELINE 40M - 5000 ITERATIONS")
print("="*70)

!python train_baseline.py

BASELINE 40M - 5000 ITERATIONS
BASELINE GPT TRAINING
Device: cuda
Dtype: bfloat16
Model: 6L, 6H, 384D
Batch size: 4 × 2 = 8
Max iterations: 5000

Loading dataset...
✗ Cache not found, tokenizing dataset (this will take ~17 minutes)...
Train examples: 1,801,350
Val examples: 3,760
Tokenizing train dataset...
Tokenizing train: 100% 1801350/1801350 [22:02<00:00, 1361.96 examples/s]
Tokenizing validation dataset...
Tokenizing val: 100% 3760/3760 [00:03<00:00, 1106.58 examples/s]
Saving tokenized dataset to cache...
✓ Cache saved!
✓ Data loaded: 450338 train batches, 940 val batches
  Data loading time: 1332.46s

Initializing model...
number of parameters: 29.96M
  Model initialization time: 0.92s
Setting up optimizer...

STARTING TRAINING

step 0: train loss 9.5449, val loss 9.5476
iter 0: loss 9.4903, time 1512.29ms, mfu -100.00%
iter 10: loss 6.0765, time 30.87ms, mfu 8.25%
iter 20: loss 3.6111, time 31.87ms, mfu 8.22%
iter 30: loss 3.6177, time 31.17ms, mfu 8.22%
iter 40: loss 2.7833, t

In [ ]:
print("="*70)
print("MOE 40M - 5000 ITERATIONS - TOP-2")
print("="*70)

!python train_moe.py

MOE 40M - 5000 ITERATIONS - TOP-2
MOE GPT TRAINING
Device: cuda
Dtype: bfloat16
Model: 6L, 6H, 384D (MoE: 8 experts, top_k=2)
Batch size: 4 × 2 = 8
Max iterations: 5000

Loading dataset...
✓ Loading tokenized dataset from cache...
Train examples: 1,801,350
Val examples: 3,760
✓ Data loaded: 450338 train batches, 940 val batches
  Data loading time: 2.65s

Initializing model...
number of parameters: 79.52M
  Model initialization time: 0.85s
Setting up optimizer...

STARTING TRAINING

step 0: train loss 9.4509, val loss 9.4521
iter 0: loss 9.4602, time 6683.18ms, mfu -100.00%
iter 10: loss 8.2459, time 56.01ms, mfu 11.51%
iter 20: loss 6.7831, time 54.65ms, mfu 11.54%
iter 30: loss 4.5626, time 54.82ms, mfu 11.57%
iter 40: loss 4.3729, time 56.91ms, mfu 11.54%
iter 50: loss 3.8943, time 54.23ms, mfu 11.58%
iter 60: loss 3.7542, time 54.43ms, mfu 11.60%
iter 70: loss 3.4937, time 53.64ms, mfu 11.65%
iter 80: loss 3.6306, time 53.88ms, mfu 11.68%
iter 90: loss 3.2508, time 55.48ms, mfu 11.

In [ ]:
# Configure 5M
!sed -i 's/n_layer = 6/n_layer = 4/g' train_baseline.py train_moe.py
!sed -i 's/n_head = 6/n_head = 4/g' train_baseline.py train_moe.py
!sed -i 's/n_embd = 384/n_embd = 128/g' train_baseline.py train_moe.py
!sed -i 's/block_size = 512/block_size = 256/g' train_baseline.py train_moe.py
!sed -i 's/max_seq_length = 512/max_seq_length = 256/g' train_baseline.py train_moe.py

!rm -rf out-baseline out-moe

print("RUNNING 5M - 5000 ITERATIONS")
!python train_baseline.py
!python train_moe.py

RUNNING 5M - 5000 ITERATIONS
BASELINE GPT TRAINING
Device: cuda
Dtype: bfloat16
Model: 4L, 4H, 128D
Batch size: 4 × 2 = 8
Max iterations: 5000

Loading dataset...
✗ Cache not found, tokenizing dataset (this will take ~17 minutes)...
Train examples: 1,801,350
Val examples: 3,760
Tokenizing train dataset...
Tokenizing train: 100% 1801350/1801350 [17:16<00:00, 1737.50 examples/s]
Tokenizing validation dataset...
Tokenizing val: 100% 3760/3760 [00:02<00:00, 1363.91 examples/s]
Saving tokenized dataset to cache...
✓ Cache saved!
✓ Data loaded: 450338 train batches, 940 val batches
  Data loading time: 1046.36s

Initializing model...
number of parameters: 7.23M
  Model initialization time: 0.33s
Setting up optimizer...

STARTING TRAINING

step 0: train loss 10.0262, val loss 10.0089
iter 0: loss 9.9492, time 1230.00ms, mfu -100.00%
iter 10: loss 9.7560, time 20.05ms, mfu 1.47%
iter 20: loss 9.4664, time 20.04ms, mfu 1.47%
iter 30: loss 8.8627, time 20.16ms, mfu 1.47%
iter 40: loss 8.9861, ti